In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool

from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import MemorySaver

/Users/shardulgore/Documents/Projects/Agentic-AI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)

True

### Asynchronous LangGraph

To run a tool:  
Sync: `tool.run(inputs)`  
Async: `await tool.arun(inputs)`

To invoke the graph:  
Sync: `graph.invoke(state)`  
Async: `await graph.ainvoke(state)`

In [3]:
class State(TypedDict):
    
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [4]:
resend_url = "https://api.resend.com/emails"

def push(text: str):
    """Send a push notification to the user"""

    # from_email = "ed@edwarddonner.com"
    to_email = "shardulggore@gmail.com"

    RESEND_API_KEY = os.environ.get("RESEND_API_KEY")
    
    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "from": f"onboarding@resend.dev",
        "to": "shardulggore@gmail.com",
        "subject": "Sales email",
        "html": f"<p>{text}</p>"  # Body wrapped in <p> tags for HTML format
    }

    requests.post(resend_url, json=payload, headers=headers)

In [5]:
tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

# tool_push.invoke("Hello, me")

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

async_browser =  create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [8]:
for tool in tools:
    print(f"{tool.name}={tool}")

click_element=async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/shardulgore/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>
navigate_browser=async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/shardulgore/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>
previous_webpage=async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/shardulgore/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>
extract_text=async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/shardulgore/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>
extract_hyperlinks=async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/shardulgore/Library/Ca

In [14]:
tool_dict = {tool.name:tool for tool in tools}

navigate_tool = tool_dict.get("navigate_browser")
extract_text_tool = tool_dict.get("extract_text")

url = 'https://edition.cnn.com/2025/06/16/tech/samsung-apple-watch-update-health'
await navigate_tool.arun({"url": url, "timeout": 60000})
text = await extract_text_tool.arun({})

TimeoutError: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://edition.cnn.com/2025/06/16/tech/samsung-apple-watch-update-health", waiting until "load"


In [12]:
import textwrap
print(textwrap.fill(text))

NameError: name 'text' is not defined

In [13]:
all_tools = tools + [tool_push]

In [ ]:
llm = ChatGroq(model="llama-3.1-8b-instant")
llm_with_tools = llm.bind_tools(all_tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
